- Importing necessary libraries
- Utility functions for Excel formatting (e.g., adjusting column width)

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np


# Excel Formating Libraries
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Font, Border, Side, Alignment
from openpyxl.utils import get_column_letter

In [3]:
def adjust_column_width(worksheet):
    for column in worksheet.columns:
        max_length = 0
        column_letter = get_column_letter(column[0].column)  # Get the column letter

        for cell in column:
            try: 
                # Check the length of the cell value
                if len(cell.value) > max_length:
                    max_length = len(cell.value)
            except:
                pass

        adjusted_width = (max_length + 2)  # Add 2 for a little extra space
        worksheet.column_dimensions[column_letter].width = adjusted_width

In [4]:
def extract_text_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup.get_text()

In [8]:
# Colors
light_greyFill = PatternFill(start_color='FFCCCCCC',
                       end_color='FFCCCCCC',
                       fill_type='solid')

#Size of the borders
thin_border = Border(left=Side(style='thin', color = '00000000'), 
                     right=Side(style='thin', color = '00000000'), 
                     top=Side(style='thin', color = '00000000'), 
                     bottom=Side(style='thin', color = '00000000'))

thick_border = Border(left=Side(style='thick', color = '00000000'), 
                     right=Side(style='thick', color = '00000000'), 
                     top=Side(style='thick', color = '00000000'), 
                     bottom=Side(style='thick', color = '00000000'))

In [9]:
df_copy = pd.read_csv('FIFA23_official_data.csv')

In [10]:
df = df_copy.copy()

In [11]:
df.head()

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Real Face,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,Release Clause,Kit Number,Best Overall Rating
0,209658,L. Goretzka,27,https://cdn.sofifa.net/players/209/658/23_60.png,Germany,https://cdn.sofifa.net/flags/de.png,87,88,FC Bayern München,https://cdn.sofifa.net/teams/21/30.png,...,Yes,"<span class=""pos pos28"">SUB","Jul 1, 2018",NaN,2026,189cm,82kg,€157M,8.0,NaN
1,212198,Bruno Fernandes,27,https://cdn.sofifa.net/players/212/198/23_60.png,Portugal,https://cdn.sofifa.net/flags/pt.png,86,87,Manchester United,https://cdn.sofifa.net/teams/11/30.png,...,Yes,"<span class=""pos pos15"">LCM","Jan 30, 2020",NaN,2026,179cm,69kg,€155M,8.0,NaN
2,224334,M. Acuña,30,https://cdn.sofifa.net/players/224/334/23_60.png,Argentina,https://cdn.sofifa.net/flags/ar.png,85,85,Sevilla FC,https://cdn.sofifa.net/teams/481/30.png,...,No,"<span class=""pos pos7"">LB","Sep 14, 2020",NaN,2024,172cm,69kg,€97.7M,19.0,NaN
3,192985,K. De Bruyne,31,https://cdn.sofifa.net/players/192/985/23_60.png,Belgium,https://cdn.sofifa.net/flags/be.png,91,91,Manchester City,https://cdn.sofifa.net/teams/10/30.png,...,Yes,"<span class=""pos pos13"">RCM","Aug 30, 2015",NaN,2025,181cm,70kg,€198.9M,17.0,NaN
4,224232,N. Barella,25,https://cdn.sofifa.net/players/224/232/23_60.png,Italy,https://cdn.sofifa.net/flags/it.png,86,89,Inter,https://cdn.sofifa.net/teams/44/30.png,...,Yes,"<span class=""pos pos13"">RCM","Sep 1, 2020",NaN,2026,172cm,68kg,€154.4M,23.0,NaN


### Data Cleaning

- Removing non-numeric characters from columns
- Converting categorical columns into ordered categories
- Handling missing values

In [13]:
for row in range(len(df['Position'])):
    if df['Position'].isna().iloc[row] == False:
        df.iloc[row, 20] = extract_text_from_html(df.iloc[row,20])

In [14]:
df['International Reputation'] = df['International Reputation'].astype(int)

In [15]:
df['Value_No'] = df['Value'].str.replace('[^\d.]','',regex = True)

In [16]:
value_no_copy = df['Value_No'].copy()

for index, value in df['Value'].items():

    if value[-1] == 'M':
        value_no_copy[index] = int(float(value_no_copy[index]) * 1000000)
    elif value[-1] == 'K':
        value_no_copy[index] = int(float(value_no_copy[index]) * 1000)

df['Value_No'] = value_no_copy

In [17]:
df['Value_No'] = df['Value_No'].astype(int)

In [18]:
df_new = df[['ID', 'Name', 'Age', 'Nationality', 'Club', 'Position','Value', 'Overall', 'Potential', 'International Reputation','Value_No']]

In [19]:
df_new = df_new[df_new['Position'].isna() != True]

In [20]:
position_order = ['GK', 'CB', 'LCB', 'RCB', 'LB', 'RB', 'LWB', 'RWB', 'CDM', 'LDM', 'RDM', 'CM', 'LCM', 'RCM', 'LM', 'RM', 'LAM', 'RAM', 'CAM', 'LW', 'RW', 'LF', 'RF', 'LS', 'RS', 'CF', 'ST', 'SUB', 'RES']

In [21]:
df_new.loc[:,'Position'] = pd.Categorical(df_new['Position'], categories = position_order, ordered = True)

df_new = df_new.sort_values(by=['Position', 'Value_No'], ascending = [True, False])

df_new.reset_index(inplace = True, drop = True)

In [23]:
position_list = df_new['Position'].unique()

- Creating aggregated position-based data
- Calculating average and sum values for each position
- Prepare the data for the Excel

In [28]:
df_new_sub = df_new.groupby('Position').agg({'Overall':'mean','Potential':'mean','International Reputation':'mean','Value_No':'sum'})
df_new_sub.reset_index(inplace = True)
df_new_sub['Position'] = df_new_sub['Position'].astype(str)
df_new_sub['Position'] = df_new_sub['Position'] + ' Subtotal'

In [29]:
len_list_position = len(position_order)
for index, item in enumerate(reversed(position_order)):
    original_index = len_list_position - 1 - index
    position_order.insert(original_index+1, f'{item} Subtotal')



In [30]:
df_new_new = pd.concat([df_new, df_new_sub])

In [31]:
df_new_new = df_new_new.fillna('')

In [32]:
df_new_new.loc[df_new_new['Position'].str.endswith(' Subtotal'), 'Value'] = round(df_new_new.loc[df_new_new['Position'].str.endswith(' Subtotal'), 'Value_No']/1000000,1)
df_new_new['Value'] = df_new_new['Value'].astype(str)
df_new_new.loc[df_new_new['Position'].str.endswith(' Subtotal'), 'Value'] = '€' + df_new_new.loc[df_new_new['Position'].str.endswith(' Subtotal'), 'Value'] + 'M'

In [33]:
df_new_new.loc[:,'Position'] = pd.Categorical(df_new_new['Position'], categories = position_order, ordered = True)

df_new_new = df_new_new.sort_values(by=['Position', 'Value_No'], ascending = [True, False])

df_new_new.reset_index(inplace = True, drop = True)

In [34]:
df_new_new.drop('Value_No', axis = 1, inplace = True)

df_new_new[['Overall','Potential','International Reputation']] = round(df_new_new[['Overall','Potential','International Reputation']],0)
df_new_new[['Overall','Potential','International Reputation']] = df_new_new[['Overall','Potential','International Reputation']].astype(int)

df_new_new['Position'] = df_new_new['Position'].astype(str)

In [35]:
df_new_new

,ID,Name,Age,Nationality,Club,Position,Value,Overall,Potential,International Reputation
0,222665.0,M. Ødegaard,23.0,Norway,Arsenal,CAM,€63.5M,84,89,3
1,189596.0,T. Müller,32.0,Germany,FC Bayern München,CAM,€63M,87,87,4
2,216594.0,N. Fekir,28.0,France,Real Betis Balompié,CAM,€54.5M,85,85,3
3,220697.0,J. Maddison,25.0,England,Leicester City,CAM,€38M,82,84,1
4,188350.0,M. Reus,33.0,Germany,Borussia Dortmund,CAM,€33.5M,85,85,4
...,...,...,...,...,...,...,...,...,...,...
17649,261630.0,21 H. Kerrido,26.0,Cameroon,,SUB,€0,60,63,1
17650,253949.0,20 R. Goss,25.0,South Africa,,SUB,€0,61,65,1
17651,252359.0,21 E. Ira Tape,22.0,Côte d'Ivoire,,SUB,€0,61,70,1
17652,168933.0,07 I. Paskov,33.0,Bulgaria,,SUB,€0,43,42,1


### Transformation in Excel

- Creating Subtotals
- Inserting subtotal rows for grouped data
- Formatting subtotal rows differently

In [37]:
num_of_rows = len(df_new_new)

In [38]:
wb = Workbook()
ws = wb.active
for r in dataframe_to_rows(df_new_new, index=False, header=True): # if index = True add one row between header and the values
    ws.append(r)

In [39]:
for name in position_list:

    start = df_new_new[df_new_new['Position'] == name].index[0]+2
    end = df_new_new[df_new_new['Position'] == name].index[-1]+2
    ws.row_dimensions.group(start, end, hidden=True)


In [40]:
# Get the position of the columns and then the column letter based in excel
list_of_columns_name = df_new_new.columns
dict_col = {}

for col in range(len(list_of_columns_name)):
    coordinate = ws[1][col].coordinate
    coordinate = coordinate.replace('1','')
    dict_col[list_of_columns_name[col]] = coordinate


In [41]:
list_strings = df_new_new.columns[:6]
list_values = df_new_new.columns[-4:]

In [42]:
# Format for all columns
for col in dict_col.values():
    for row in range(num_of_rows+1):
        ws[col+str(row+1)].border = thin_border

In [43]:
# Format for countries
for col in list_strings:
    for row in range(num_of_rows+1):
        #ws[dict_col[col]+str(row+1)].font = Font(bold = True)
        ws[dict_col[col]+str(row+1)].alignment  = Alignment(horizontal='left')
        
        if ws[dict_col['Position']+str(row+1)].value.endswith('Subtotal') == True:
            ws[dict_col[col]+str(row+1)].font = Font(bold = True)

In [44]:
# Format for values
for col in list_values:
    for row in range(num_of_rows+1):
        #ws[dict_col[col]+str(row+1)].font = Font(bold = True)
        ws[dict_col[col]+str(row+1)].alignment  = Alignment(horizontal='right')
        #ws[dict_col[col]+str(row+1)].number_format = '€#,##0.00'
        
        if ws[dict_col['Position']+str(row+1)].value.endswith('Subtotal') == True:
            ws[dict_col[col]+str(row+1)].font = Font(bold = True)

In [45]:
# Format for headers
for col in dict_col.values():
    ws[col+str(1)].font = Font(bold = True)
    ws[col+str(1)].alignment  = Alignment(horizontal='center', vertical = 'center', wrapText = True)
    ws[col+str(1)].border = thick_border
    ws[col+str(1)].fill = light_greyFill

In [46]:
adjust_column_width(ws)

In [47]:
wb.save( "ManipulationAndSubotals.xlsx")